In [ ]:
import json

with open("sotopia_pi_bc_episodes_reward_goal_progress_gpt-4o.json", 'r') as f:
    data = json.load(f)

print(len(data))
print(data[0].keys())

new_data = []
for d in data:
    if d['value'] > 0:
        new_data.append(d)

with open("sotopia_pi_bc_episodes_reward_goal_progress_gpt-4o_only_pos.json", 'w') as f:
    data = json.dump(new_data, f)
print(len(new_data))

In [ ]:
import json
from collections import Counter
with open("sotopia_pi_bc_episodes_reward_only_response_gpt-4o.json", 'r') as f:
    data = json.load(f)

print(len(data))
print(data[0].keys())
print(Counter([d['value'] for d in data]))
new_data = []
for d in data:
    if d['value'] > 0:
        new_data.append(d)

with open("sotopia_pi_bc_episodes_reward_only_response_gpt-4o_only_pos.json", 'w') as f:
    data = json.dump(new_data, f)
print(len(new_data))